<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/GroceryData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install faker

     |████████████████████████████████| 1.5 MB 4.7 MB/s 


In [68]:
!wget -q https://github.com/profteachkids/StemUnleashed/raw/main/GroceryPrices.xlsx -O GroceryPrices.xlsx

In [63]:
import requests
from lxml import html
import numpy as np
import pandas as pd
from collections import deque
from faker import Faker

In [252]:
idx=pd.IndexSlice

In [64]:
rng=np.random.RandomState()

In [69]:
df_grocery = pd.read_excel('GroceryPrices.xlsx')

In [169]:
fake=Faker()
names=[fake.name_nonbinary() for i in range(100)]

In [263]:
now = pd.to_datetime('now').round('s')
df_names=[]
for _ in range(100):
    name = rng.choice(names)
    starts = now + pd.to_timedelta(rng.randint(100,24*3600*30,size=rng.poisson(3)+1),'s')
    df_start=[]
    for start in starts:
        n_items = rng.poisson(10)+1
        times = start + pd.to_timedelta(np.cumsum(rng.poisson(5,n_items))+1,'s')
        items = rng.choice(df_grocery['Item'],n_items)
        quantity = rng.poisson(0.5, size=n_items)+1
        df_start.append(pd.DataFrame(dict(time=times, item=items,quantity=quantity)))

    df_name=pd.concat(df_start)
    df_name['name']=name
    df_names.append(df_name)
df_shopping=pd.concat(df_names)

In [268]:
df_shopping2=df_shopping.sort_values(by=['name','time'])

In [293]:
def trip_count(d):
    d['trip']=np.cumsum(d['time'].diff()>pd.to_timedelta(60,'s'))
    return d 

In [297]:
df_trip = df_shopping2.groupby('name').apply(trip_count)

In [311]:
def checkout_duration(d):
    return (pd.Timedelta(d['time'].max()-d['time'].min()))/pd.to_timedelta(1,'s')

In [313]:
df_checkout_duration=df_trip.groupby(['name','trip']).apply(checkout_duration)

In [314]:
df_checkout_duration

name             trip
Amanda Brown     0       47.0
                 1       50.0
                 2       32.0
                 3       65.0
                 4       48.0
                         ... 
Yesenia Edwards  9       20.0
                 10      36.0
                 11      46.0
                 12      43.0
                 13      66.0
Length: 421, dtype: float64